# Analysis of New York Motor Vehicle Collisions
### UCDPA Project, Hauke Laing

## Setup

In [36]:
# import required libraries
import requests
import pandas as pd
import numpy as np
import re
import seaborn as sns

## Data Collection

### helper functions

In [37]:
def read_api_chunk(api, limit=1000, offset=0):
    """read a single chunk from the api"""
    return pd.read_json(f"{api}?${limit=}&${offset=}")


def read_api(api, total_size, chunk_size=1000):
    """read given number of lines from api, applying the chunk_size along the way"""
    # use https://docs.python.org/3/reference/expressions.html#yield-expressions
    chunk_generator = (
        # define chunks; the last chunk might be smaller than chunk_size
        read_api_chunk(api, limit=min(chunk_size, total_size - x), offset=x)
        for x in range(0, total_size, chunk_size)
    )
    # in the generator expressions, the chunks are not yet read and stored in memory
    # the outer paranthesis are synctactilly required for generator expressions; they
    # are not included simply in order to permit the multiline definition

    # pd.concat can handle generator expressions. According to the api reference, the objs argument
    # accepts a sequence of DataFrame objects. This indicates that any iterable that yields DataFrame
    # objects will be accepted, which is what chunk_generator provides.
    return pd.concat(chunk_generator)

### inputs

In [38]:
# set input parameters
api = "https://data.cityofnewyork.us/resource/h9gi-nx95.json"
n = 50e3
# limit = 1000

### read data

In [39]:
# read data
# data_raw = read_api_chunk(api, limit=int(n))
data_raw = read_api(api, total_size=int(100e3), chunk_size=int(25e3))

## Data Preparation 

### Cleaning

In [96]:
# initial cleaning of data
data_raw_2 = data_raw.rename(
    columns={
        "vehicle_type_code1": "vehicle_type_code_1",
        "vehicle_type_code2": "vehicle_type_code_2",
    }
)
data_raw_2 = data_raw_2.reindex(sorted(data_raw_2.columns), axis=1)
text_cols = [col for col in data_raw_2 if re.search("(street|contributing_factor)", col)]
data_raw_2[text_cols] = data_raw_2[text_cols].astype("string")
data_raw_2.shape

borough                                  object
collision_id                              int64
contributing_factor_vehicle_1    string[python]
contributing_factor_vehicle_2    string[python]
contributing_factor_vehicle_3    string[python]
contributing_factor_vehicle_4    string[python]
contributing_factor_vehicle_5    string[python]
crash_date                               object
crash_time                       datetime64[ns]
cross_street_name                string[python]
latitude                                float64
location                                 object
longitude                               float64
number_of_cyclist_injured                 int64
number_of_cyclist_killed                  int64
number_of_motorist_injured                int64
number_of_motorist_killed                 int64
number_of_pedestrians_injured             int64
number_of_pedestrians_killed              int64
number_of_persons_injured                 int64
number_of_persons_killed                

In [98]:
# set index
data_raw_2.set_index(keys="collision_id", drop=False, inplace=True)
# ensure index is unique
data_raw_2 = data_raw_2[~data_raw_2.index.duplicated(keep='first')]

In [99]:
# plausibility check
invalid_number_of_injured = (
    data_raw_2[
        [
            "number_of_pedestrians_injured",
            "number_of_cyclist_injured",
            "number_of_motorist_injured",
        ]
    ].sum(axis=1)
    > data_raw_2["number_of_persons_injured"]
)
sum(invalid_number_of_injured)

0

In [196]:
# add column to indicate if a cyclist was injured
data_raw_2.assign(cyclist_was_injured=lambda x: x.number_of_cyclist_injured > 0)
data_raw_2['cyclist_was_injured']

collision_id
4455765    False
4513547    False
4541903    False
4456314    False
4486609    False
           ...  
4545240    False
4521103    False
4545395    False
4502444     True
4502508    False
Name: cyclist_was_injured, Length: 99989, dtype: bool

### Explore

In [197]:
# explore dataset
occ = data_raw_2["number_of_cyclist_injured"].value_counts()
print(occ)

number_of_cyclist_injured
0    95410
1     4508
2       66
3        5
Name: count, dtype: int64


In [198]:
print(occ / sum(occ))

number_of_cyclist_injured
0    0.954205
1    0.045085
2    0.000660
3    0.000050
Name: count, dtype: float64


### Wrangle

In [199]:
# the dataset has 5 columns to cover up to 5 vehicles involved in a collision
# format data to cover 5 vehicles in one column
data_long = pd.wide_to_long(
    data_raw_2,
    stubnames=["vehicle_type_code_", "contributing_factor_vehicle_"],
    i="collision_id",
    j="vehicle_no",
)
data_long.rename(
    columns={
        "vehicle_type_code_": "vehicle_type_code",
        "contributing_factor_vehicle_": "contributing_factor_vehicle",
    },
    inplace=True,
)
data_long = data_long.reindex(sorted(data_long.columns), axis=1)


# keep rows for vehicle no. > 1 only if relevant information pertaining to the vehicle is present; the row is redundant otherwise
_cnd1 = (
    data_long[["vehicle_type_code", "contributing_factor_vehicle"]]
    .notnull()
    .any(axis=1)
)
_cnd2 = data_long.index.get_level_values(level=1) == 1
_cnd = _cnd1 | _cnd2
data_long = data_long.loc[_cnd, :]
# export long data
# data_long.to_csv("data_long.csv")
data_long.dtypes

borough                                  object
contributing_factor_vehicle      string[python]
crash_date                               object
crash_time                       datetime64[ns]
cross_street_name                string[python]
cyclist_was_injured                        bool
latitude                                float64
location                                 object
longitude                               float64
number_of_cyclist_injured                 int64
number_of_cyclist_killed                  int64
number_of_motorist_injured                int64
number_of_motorist_killed                 int64
number_of_pedestrians_injured             int64
number_of_pedestrians_killed              int64
number_of_persons_injured                 int64
number_of_persons_killed                  int64
off_street_name                  string[python]
on_street_name                   string[python]
vehicle_type_code                        object
zip_code                                

In [105]:
# explore the column 'contributing_factor_vehicle'
data_long["contributing_factor_vehicle"].value_counts()

contributing_factor_vehicle
Unspecified                                          104193
Driver Inattention/Distraction                        28036
Following Too Closely                                  7869
Failure to Yield Right-of-Way                          7589
Passing or Lane Usage Improper                         5268
Unsafe Speed                                           4239
Passing Too Closely                                    4226
Other Vehicular                                        3980
Traffic Control Disregarded                            3418
Backing Unsafely                                       3285
Turning Improperly                                     2485
Unsafe Lane Changing                                   2351
Driver Inexperience                                    2282
Alcohol Involvement                                    1643
Reaction to Uninvolved Vehicle                         1550
Pedestrian/Bicyclist/Other Pedestrian Error/Co...      1225
View Obstruc

In [200]:
# The column 'contributing_factor_vehicle' contains a text comment. To prepare the column 
# for machine learning algorithms, we want to categorize it and later create dummies

# First, establish a code representing a contributing factor and a corresponding mapping

confac = (
    data_long["contributing_factor_vehicle"]
    .drop_duplicates()
    .dropna()
    .reset_index(drop=True)
    .to_frame(name="contributing_factor")
)

def get_first_chars(input):
    """retrieve first character of each word in a string of words"""
    return "".join(item[0].upper() for item in re.findall("\w+", input))

# create initial code
confac["cf"] = confac["contributing_factor"].apply(get_first_chars)

# if code is not unique, add counting index
confac["n"] = confac.groupby(["cf"]).cumcount()
_k = confac["n"] > 0
confac.loc[_k, "cf"] = confac.loc[_k, "cf"] + confac.loc[_k, "n"].astype("string")
confac.set_index("contributing_factor", inplace=True)

# export mapping for reference
confac.to_csv("output/confac.csv")

confac_cols = "cf." + confac["cf"]
mapping_cf = pd.Series(confac["cf"]).to_dict()

In [111]:
# determine dummies grouped by collision_id
# data_long["cf"] = data_long["contributing_factor_vehicle"].replace(mapping_cf)
# dummies_cf_long = pd.get_dummies(data_long, columns=["cf"], prefix_sep=".")
# dummies_cf = dummies_cf_long[confac_cols].groupby(level=0).max()
# dummies_cf["n_vehicles"] = dummies_cf.sum(axis=1) # store number of vehicles involved
# dummies_cf.head()

In [201]:
# similar to the column 'contributing factor' is the column 'vehicle type'
# we want to prepare it as categorical data

# vehicle_type_codes = data_long["vehicle_type_code"].astype("string").dropna()

# align formatting of text
data_long["vehicle_type"] = (
    data_long["vehicle_type_code"]
    .str.replace(pat=r"\W+", repl="_", regex=True)
    .str.lower()
)

print(data_long["vehicle_type"].nunique())
print(data_long["vehicle_type"].value_counts())

429
vehicle_type
sedan                                  82778
station_wagon_sport_utility_vehicle    60668
bike                                    4948
pick_up_truck                           3874
box_truck                               3841
                                       ...  
citywide                                   1
pro_master                                 1
rgr                                        1
us_postal                                  1
c3                                         1
Name: count, Length: 429, dtype: int64


In [202]:
# reduce to top 50 most frequent types of cars. last category is set to "other"
top = 50
vehicle_top_cats = data_long["vehicle_type"].value_counts().head(top - 1).index
data_long["vt"] = data_long["vehicle_type"]

data_long.loc[
    ~data_long["vehicle_type"].isin(vehicle_top_cats)
    & ~data_long["vehicle_type"].isna(),
    ["vt"],
] = "other"

data_long["vt"] = data_long["vt"].astype("category")
data_long["vt"].value_counts()

vt
sedan                                  82778
station_wagon_sport_utility_vehicle    60668
bike                                    4948
pick_up_truck                           3874
box_truck                               3841
taxi                                    3717
bus                                     3050
e_bike                                  2735
motorcycle                              1715
tractor_truck_diesel                    1431
e_scooter                               1385
van                                     1205
ambulance                               1050
other                                    869
moped                                    686
dump                                     628
pk                                       427
flat_bed                                 363
garbage_or_refuse                        354
convertible                              342
carry_all                                262
motorscooter                             240
tow_tru

In [203]:
# now we are ready to create dummies from the 'contributing factor' and
# 'vehicle type' columns. The processed versions of these columns have been 
# stored as 'cf' and 'vt'

dummies_long = pd.get_dummies(
    data_long, prefix=["vt", "cf"], columns=["vt", "cf"], prefix_sep="."
)
dummies = dummies_long.filter(regex=r"^(vt|cf)\.").groupby(level=0).max()
dummies.head()

KeyError: "['cf'] not in index"

In [204]:
# after creating dummies, it is useful to create a copy of
# the data with the source columns for the dummies removed
data_raw_3 = data_raw_2.drop(
    columns=[
        "vehicle_type_code_1",
        "vehicle_type_code_2",
        "vehicle_type_code_3",
        "vehicle_type_code_4",
        "vehicle_type_code_5",
        "contributing_factor_vehicle_1",
        "contributing_factor_vehicle_2",
        "contributing_factor_vehicle_3",
        "contributing_factor_vehicle_4",
        "contributing_factor_vehicle_5",
    ]
)

In [205]:
collisions = data_raw_3.join(dummies)
collisions.to_csv("output/collisions.csv")

## Algorithm Setup

In [206]:
# import performance metrix
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html#sklearn.metrics.accuracy_score
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html#sklearn-metrics-confusion-matrix
from sklearn.metrics import accuracy_score, confusion_matrix

# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html#sklearn.model_selection.train_test_split
from sklearn.model_selection import train_test_split

# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html
# https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation
from sklearn.model_selection import cross_val_score


In [229]:
# split data
X = collisions.filter(regex=r"^(vt|cf)\.", axis=1)
y = collisions["cyclist_was_injured"]
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [230]:
print(y.value_counts())

cyclist_was_injured
False    95410
True      4579
Name: count, dtype: int64


In [260]:
from sklearn.tree import DecisionTreeClassifier


In [264]:

# https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html
dt1 = DecisionTreeClassifier(criterion='gini')
dt2 = DecisionTreeClassifier(criterion='entropy')
dt3 = DecisionTreeClassifier(max_depth=2)
# dt.fit(X_train, y_train)
# dt.score(X_test, y_test)
print(cross_val_score(dt1, X, y))
print(cross_val_score(dt2, X, y))
print(cross_val_score(dt3, X, y))


[0.98054805 0.98089809 0.97979798 0.97984798 0.98024704]
[0.98059806 0.98109811 0.979998   0.97979798 0.98024704]
[0.98189819 0.98249825 0.98289829 0.98234823 0.98204731]


In [259]:
y_dt = dt.predict(X_test)
confusion_matrix(y_test, y_dt)

array([[23586,   268],
       [  209,   935]], dtype=int64)

In [149]:
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0)
iris = load_iris()
cross_val_score(clf, iris.data, iris.target, cv=10)
cross_val_score(clf, iris.data, iris.target)

array([0.96666667, 0.96666667, 0.9       , 0.96666667, 1.        ])

In [43]:
# test ridge classifier
from sklearn.linear_model import RidgeClassifier

# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeClassifier.html#sklearn.linear_model.RidgeClassifier
clf_ridge = RidgeClassifier()
clf_ridge.fit(X_train, y_train)
y_pred = clf_ridge.predict(X_test)
clf_ridge.score(X_test, y_test)
# accuracy_score(y_pred, y_test)

0.71528

In [15]:
# test random tree classifier
from sklearn.ensemble import RandomForestClassifier

# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn-ensemble-randomforestclassifier
clf_rfc = RandomForestClassifier(n_estimators=10)
# clf_rfc = clf_rfc.fit(X_train, y_train)
clf_rfc.fit(X_train, y_train)
clf_rfc.score(X_test, y_test)

0.71464

In [17]:
from sklearn.neighbors import KNeighborsClassifier

# https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn-neighbors-kneighborsclassifier
clf_knn = KNeighborsClassifier(algorithm="ball_tree")
clf_knn = clf_knn.fit(X_train, y_train)
clf_knn.score(X_test, y_test)

0.69096

## Visualisation

In [ ]:
# https://campus.datacamp.com/courses/supervised-learning-with-scikit-learn/preprocessing-and-pipelines-4?ex=14
# # Create models dictionary
# models = {
#     "Logistic Regression": LogisticRegression(),
#     "KNN": KNeighborsClassifier(),
#     "Decision Tree Classifier": DecisionTreeClassifier(),
# }
# results = []

# # Loop through the models' values
# for model in models.values():
#     # Instantiate a KFold object
#     kf = KFold(n_splits=6, random_state=12, shuffle=True)

#     # Perform cross-validation
#     cv_results = cross_val_score(model, X_train_scaled, y_train, cv=kf)
#     results.append(cv_results)
# plt.boxplot(results, labels=models.keys())
# plt.show()

In [39]:
from sklearn.pipeline import make_pipeline, Pipeline

pipe = make_pipeline([clf_ridge, clf_rfc, clf_dct])
# pipe.fit(X_train, y_train)

TypeError: Last step of Pipeline should implement fit or be the string 'passthrough'. '[RidgeClassifier(), RandomForestClassifier(n_estimators=10), DecisionTreeClassifier(min_samples_leaf=3, min_samples_split=9,
                       random_state=500)]' (type <class 'list'>) doesn't

In [33]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier

# # Prepare the list of tuples with the first-layer classifiers


clf_ridge = RidgeClassifier()
clf_rfc = RandomForestClassifier(n_estimators=10)
clf_dct = DecisionTreeClassifier(
    min_samples_leaf=3, min_samples_split=9, random_state=500
)

classifiers = [clf_ridge, clf_rfc, clf_dct]

estimators = [
    # ('ridge', RidgeClassifier()),
    ("random_forest", RandomForestClassifier(n_estimators=10)),
    (
        "decision_tree",
        DecisionTreeClassifier(
            min_samples_leaf=3, min_samples_split=9, random_state=500
        ),
    ),
]

# Instantiate the second-layer meta estimator
clf_meta = LogisticRegression()
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression

# Build the stacking classifier
# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.StackingClassifier.html#sklearn-ensemble-stackingclassifier
clf_stack = StackingClassifier(
    estimators=estimators,
    final_estimator=clf_meta,
    # stack_method='predict_proba',
    passthrough=False,
)


clf_stack.fit(X_train, y_train)
clf_stack.score(X_test, y_test)

c:\Users\hauke\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
c:\Users\hauke\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py:1166: RuntimeWarning: Number of classes in training fold (12) does not match total number of classes (13). Results may not be appropriate for your use case. To fix this, use a cross-validation technique resulting in properly stratified folds
  warnings.warn(
c:\Users\hauke\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py:1166: RuntimeWarning: Number of classes in training fold (12) does not match total number of classes (13). Results may not be appropriate for your use case. To fix this, use a cross-validation technique resulting in properly stratified folds
  warnings.warn(
c:\Users\hauke\AppData\Local\Progr

0.71872

In [28]:
zip(*classifiers)

TypeError: 'RidgeClassifier' object is not iterable

In [ ]:
clf_rfc.get_params()

## Roads

In [ ]:
# data["on_street_name"].str.split(" ")

## Cyclists

### Injured Cyclists

In [ ]:
data["number_of_cyclist_injured"].value_counts(dropna=True)
# data.head(10)

### Contributing factors